In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [33]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

[(['Yo', 'creo', 'que', 'si'], 'SPANISH'),
 (['it', 'is', 'lost', 'on', 'me'], 'ENGLISH')]

In [3]:
word_to_ix = {}
for sent,_ in data+test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [4]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [6]:
class BOW_Classifier(nn.Module):
    def __init__(self,num_labels,vocab_size):
        super(BOW_Classifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
    def forward(self,bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

In [19]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1,-1)

In [20]:
def make_target (label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [21]:
model = BOW_Classifier(NUM_LABELS,VOCAB_SIZE)

In [22]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.0882, -0.1489, -0.1825,  0.0093, -0.0244,  0.1287, -0.0568,
         -0.0544, -0.1505, -0.1322, -0.0510,  0.0028,  0.1181,  0.1086,
         -0.1329,  0.0894, -0.0130,  0.1523,  0.1094, -0.1718, -0.0796,
          0.0254,  0.1699, -0.0980,  0.1535,  0.0530],
        [ 0.1912, -0.0181, -0.1199, -0.1911, -0.0176,  0.0181, -0.0325,
         -0.0277,  0.0443, -0.1791,  0.1863, -0.0439,  0.1303, -0.1573,
          0.1435, -0.1231,  0.1616, -0.0453, -0.0234,  0.1757,  0.0450,
         -0.1395,  0.1198, -0.1296,  0.1161,  0.0428]])
Parameter containing:
tensor([-0.1793, -0.0396])


In [36]:
with torch.no_grad():
    sample = data[1]
    bow_vector = make_bow_vector(sample[0], word_to_ix)
    log_probs = model(bow_vector)
    print(log_probs)

tensor([[-1.0799, -0.4150]])


In [32]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

In [38]:
with torch.no_grad():
    for instance,label in test_data:
        bow_vector = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vector)
        print(log_probs)

tensor([[-0.8323, -0.5710]])
tensor([[-0.9403, -0.4951]])


In [39]:
print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([-0.0510,  0.1863])


In [44]:
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [46]:
for epoch in range(100):
    for instance, label in data:
        model.zero_grad()
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)
        log_probs = model(bow_vec)
        loss = loss_function(log_probs,target)
        loss.backward()
        optimizer.step()
        
with torch.no_grad():
    for instance, label in test_data:
        bow_vector = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vector)
        print(log_probs)

print(next(model.parameters())[:, word_to_ix["creo"]])

tensor([[-0.1568, -1.9299]])
tensor([[-2.7577, -0.0655]])
tensor([ 0.3809, -0.2457])
